In [1]:

import re
from pathlib import Path
from typing import List, Tuple


from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaLLM
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
import ollama

from pathlib import Path
import pdf2image
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\mathe\repos\local\vinho\Tesseract-OCR\tesseract.exe"


In [2]:
data_folder = Path("data")
pdf_files = list(data_folder.glob("*.pdf"))
csv_files = list(data_folder.glob("*.csv"))
pages_description = []
pages_matches = []


In [3]:

def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file, poppler_path=r"C:\Users\mathe\repos\local\vinho\poppler\poppler-24.08.0\Library\bin")

def ocr_core(file):
    return pytesseract.image_to_string(file)


In [4]:
for pdf_file in pdf_files:
    images = pdf_to_img(str(pdf_file))
    for pg, img in enumerate(images):
        text = ocr_core(img)
        if text.strip():
            pages_description.append(Document(page_content=text, metadata={"source": str(pdf_file), "page": pg + 1}))


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [ ]:
for csv_file in csv_files:
    loader = CSVLoader(str(csv_file), encoding="utf-8")
    csv_pages = loader.load()
    pages_matches.extend(csv_pages)


In [ ]:
def clean_text(texto: str) -> str:
    texto = re.sub(r"(?<!\n)\n(?!\n)", " ", texto)
    return texto.strip()


In [ ]:
for page in pages_description:
    page.page_content = clean_text(page.page_content)

for page in pages_matches:
    page.page_content = clean_text(page.page_content)


In [ ]:
print(pages_description[90].page_content)


PENINSULA DE SETUBAL

OS VINHOS

Foram os Fenicios e os Gregos que trouxeram do Préximo Oriente muitas castas para esta regido. Perceberam que o clima ameno, as encostas da Arradbida e a zona ribeirinha do Tejo eram propicias ao cultivo da vinha.

Ao longo dos_ séculos outros povos promoveram a produgaéo de vinho na regiao, tradigao que ainda hoje prevalece.

A Peninsula de Setubal varia entre zonas planas e arenosas e a paisagem mais montanhosa da Serra da Arradbida.

E é aqui perto da Serra da Arrabida, com solos de argilo-calcdrio e clima suave, que se conjugam as condigées perfeitas para produzir vinhos D.O. Setubal, que inclui os tradicionais vinhos fortificados Moscatel de Setubal e Moscatel Roxo de Setubal.

Este clima e restantes condigées também é favordvel a produgao de vinhos brancos de alta qualidade, com frescura, vigor e capacidade de envelhecimento.

Nas planicies com solos arenosos e grandes amplitudes térmicas, ha 6timas condigdes para produzir vinhos tintos robustos, 

In [ ]:

docs_description = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(pages_description)
docs_matches = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=200).split_documents(pages_matches)



In [ ]:
len(docs_description), len(docs_matches)


(232, 122)

In [ ]:
docs_matches[10].page_content


'\ufeffname: Vinho Rosé Góes Tempos Pétalas Cabernet Franc 2023 short_description: Delicados aromas de frutas vermelhas e toques florais, perfeito para massas, risotos e carnes brancas. pix_total: 62.4 image_src: https://cdn.dooca.store/158171/products/dne7b7kp0yy0dggetftbwuegpfjjz4kbonzf.jpg?v=1722640506'

In [ ]:
docs_description[91].page_content


'A GESTAO DO BEVERAGE COST, OU CUSTO DAS BEBIDAS, E UMA PARTE FUNDAMENTAL DA GESTAO FINANCEIRA DE UM NEGOCIO DE RESTAURAGAO.\n\nO Beverage Cost refere-se ao controle dos custos associados as bebidas servidas no estabelecimento, incluindo alcool, refrigerantes, sumos e outras bebidas.\n\nAqui estao algumas praticas comuns na gestao do beverage cost:\n\nControlo de Stock: Manter um _ controlo minucioso do inventdrio de bebidas é essencial para gerir o beverage cost. Isso inclui registar todas as entradas e saidas de bebidas, supervisionar o nivel de stock regularmente e identificar possiveis perdas ou desperdicios.\n\nPregos do Menu: Definir pregos adequados para as bebidas no menu é€ crucial para garantir que o- restaurante obtenha rentabilidade. Isso envolve considerar os custos de compra das bebidas, margens de lucro desejadas e pregos competitivos no mercado.\n\n36'

In [ ]:
for doc in docs_description:
    doc.page_content = "passage: " + doc.page_content
    
for doc in docs_matches:
    doc.page_content = "passage: " + doc.page_content
    


In [ ]:
docs_description[91].page_content


'passage: A GESTAO DO BEVERAGE COST, OU CUSTO DAS BEBIDAS, E UMA PARTE FUNDAMENTAL DA GESTAO FINANCEIRA DE UM NEGOCIO DE RESTAURAGAO.\n\nO Beverage Cost refere-se ao controle dos custos associados as bebidas servidas no estabelecimento, incluindo alcool, refrigerantes, sumos e outras bebidas.\n\nAqui estao algumas praticas comuns na gestao do beverage cost:\n\nControlo de Stock: Manter um _ controlo minucioso do inventdrio de bebidas é essencial para gerir o beverage cost. Isso inclui registar todas as entradas e saidas de bebidas, supervisionar o nivel de stock regularmente e identificar possiveis perdas ou desperdicios.\n\nPregos do Menu: Definir pregos adequados para as bebidas no menu é€ crucial para garantir que o- restaurante obtenha rentabilidade. Isso envolve considerar os custos de compra das bebidas, margens de lucro desejadas e pregos competitivos no mercado.\n\n36'

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")


In [ ]:
db_description = FAISS.from_documents(docs_description, embedding)
db_matches = FAISS.from_documents(docs_matches, embedding)

db_description.save_local("./faiss")
db_matches.save_local("./faiss")


In [ ]:
MODEL = "mistral:instruct"


In [ ]:
ollama.pull(MODEL)


ProgressResponse(status='success', completed=None, total=None, digest=None)

In [ ]:
llm = OllamaLLM(model=MODEL)


In [ ]:
def get_answer(question: str) -> str:
    return llm.invoke(question)


In [ ]:
# db = FAISS.load_local("faiss", embedding, allow_dangerous_deserialization=True)
retriever_description = db_description.as_retriever(search_kwargs={"k": 3})
retriever_matches = db_matches.as_retriever(search_kwargs={"k": 3})


In [ ]:
def formatar_resposta(texto):
    texto = texto.replace(". ", ".\n")
    texto = texto.replace(" -", "\n-")
    return texto.strip()


# Descricao


In [ ]:
pergunta = "Qual vinho melhor combina com stake tartar"


In [ ]:
role = """Você é um sommelier experiente.
          Sua função é recomendar vinhos assertivos ao cliente
          voce acabou de receber um prato, e deve usar apenas o contexto fornecido para a tarefa,
          descreva brevemente como deveria ser o vinho ideal para esse prato,
          falando sobre o tipo de vinho, cor, aroma, corpo, acidez, etc,
          seja sucinto e em poucas palavras
          exemplo: 
          "Vinho Branco, acididade leve e aromas frutados que harmonizam bem com o sabor cremoso do queijo"
          "Vinho Verde, fresco e frutado, combina bem com a doçura e acidez da morango. 
          "vinho branco leve, com aromas citrus ou frutados, de corpo médio, para que a saborosa carne do camarão se destaque."
          """


In [ ]:
# role = """
# Você é um sommelier especialista em vinhos brasileiros, com conhecimento profundo sobre harmonização gastronômica. Sua função é fornecer recomendações precisas e educativas sobre vinhos, baseando-se nas informações recuperadas do banco de dados de vinhos.

# Restrições:
# - Baseie-se apenas nas informações recuperadas do banco de dados
# - Não invente características de vinhos não presentes nos dados
# - Se não houver informação suficiente, seja honesto sobre as limitações
# - Priorize vinhos brasileiros da base de dados
# - Mantenha recomendações dentro do contexto dos vinhos disponíveis
# """


In [ ]:
query = "query: " + pergunta
docs_relevantes = retriever_description.get_relevant_documents(query)


docs_relevantes


[Document(id='24432ea7-fc2b-437a-b59a-73acb60f75db', metadata={'source': 'data\\MANUAL-CARTAS-DE-VINHO_2024-ADREPES.pdf', 'page': 88}, page_content='passage: A textura macia e cremosa da Torta de Azeitaéo, com seu sabor doce e aromatico, harmoniza perfeitamente com o vinho licoroso Moscatel de Setubal. A dogura e densidade do Moscatel equilibram a consisténcia e o doce da torta. E recomendavel optar por vinhos de perfil mais jovem, onde notas de laranja, mel, alperce e tamara se destacam. Um bom nivel de acidez é essencial para manter a harmonia e evitar que a combinagao se torne enjoativa. Servir o vinho a uma temperatura de 10°C-12°C potencializa essa harmonia. Vinhos doces de colheita tardia ou espumantes doces também podem surpreender de forma positiva.'),
 Document(id='ced1ada1-5e30-44d7-8472-d11b58408647', metadata={'source': 'data\\MANUAL-CARTAS-DE-VINHO_2024-ADREPES.pdf', 'page': 80}, page_content='passage: Embora os vinhos tintos sejam geralmente os mais competentes, alguns vi

In [ ]:
contexto = ". ".join([doc.page_content for doc in docs_relevantes])


In [113]:
print(formatar_resposta(contexto))


passage: A textura macia e cremosa da Torta de Azeitaéo, com seu sabor doce e aromatico, harmoniza perfeitamente com o vinho licoroso Moscatel de Setubal.
A dogura e densidade do Moscatel equilibram a consisténcia e o doce da torta.
E recomendavel optar por vinhos de perfil mais jovem, onde notas de laranja, mel, alperce e tamara se destacam.
Um bom nivel de acidez é essencial para manter a harmonia e evitar que a combinagao se torne enjoativa.
Servir o vinho a uma temperatura de 10°C-12°C potencializa essa harmonia.
Vinhos doces de colheita tardia ou espumantes doces também podem surpreender de forma positiva..
passage: Embora os vinhos tintos sejam geralmente os mais competentes, alguns vinhos brancos entroncados de corpo, conseguem lidar com a exigéncia dos sabores do prato e enfrentar a substancia e firmeza da carne, assim como a envolvéncia do caldo.
A incluséo de alguns elementos que acrescentem frescura, tal como citricos e ervas aromaticas, podem facilitar a vida ao vinho branc

In [114]:
prompt = f"""
<role>
{role}
</role>
<contexto>:
{contexto}
</contexto>
<pergunta>:
{pergunta}
</pergunta>
<resposta>:
...
"""


In [115]:
description = get_answer(prompt)


In [116]:
description


' Vinho tinto de corpo firme e estrutura bem definida, aromas de fruto seco ou herbáceos, boa carga de taninos para contrapôr a graça da carne. A acididade equilibrada traz frescura e harmonia ao paladar. Servir entre 16°C-18°C.\n\nExemplo: Aragonés ou Grand Noir, provenientes do Norte Alentejo, têm personalidade e equilibrio, além de um potencial para longevidade.'

In [126]:

role = """Você é um sommelier experiente.
          trabalha em um renomado restaurante 5 estrelas
          Sua função é recomendar vinhos assertivos ao cliente
          voce acabou de receber uma descrição de vinho, e deve usar o contexto para construir uma recomendação apropriada,
          lembre que voce esta falando diretamente com o cliente, entao seja educado e profissional,
          também explique o porque o vinho escolhido é o melhor para o cliente, e como ele orna com o prato
          também forneça seu preço"""


In [127]:
query = "query: " + description
docs_relevantes = retriever_matches.get_relevant_documents(query)

docs_relevantes


[Document(id='b2656b73-57cc-462a-af29-3078742d6923', metadata={'source': 'data\\vinhos_completos.csv', 'row': 12}, page_content='passage: \ufeffname: Vinho Tinto Tempos de Góes Tannat Reserva 2023 short_description: Aromas de frutas vermelhas e especiarias, taninos firmes, ideal para grelhados e queijos fortes. É ideal para ser servido entre 16ºC a 18ºC, harmonizando perfeitamente com grelhados, carnes exóticas, queijos fortes e molhos condimentados. pix_total: 94.08 image_src: https://cdn.dooca.store/158171/products/0jku3pabhtem1houcu6nvk7rjbkcz5q3ar6o.jpg?v=1722526922'),
 Document(id='92bbc360-b6c6-4a0b-86d6-1dca2e3a5d7c', metadata={'source': 'data\\vinhos_completos.csv', 'row': 21}, page_content='passage: \ufeffname: Vinho Tinto Góes Tradição Bordô Suave short_description: Aromas intensos e paladar suave com a profundidade do rubi, ideal para carnes, massas e queijos. pix_total: 19.2 image_src: https://cdn.dooca.store/158171/products/stiosdee553pxevisvdlarz7tandsboelvuo.jpg?v=172348

In [128]:
# re.sub(r'[^a-zA-Z0-9\s]', '',
contexto = ". ".join([re.sub(r'\b\w+:\s*', '', doc.page_content) for doc in docs_relevantes])


In [129]:
print(formatar_resposta(contexto))


Vinho Tinto Tempos de Góes Tannat Reserva 2023 Aromas de frutas vermelhas e especiarias, taninos firmes, ideal para grelhados e queijos fortes.
É ideal para ser servido entre 16ºC a 18ºC, harmonizando perfeitamente com grelhados, carnes exóticas, queijos fortes e molhos condimentados.
94.08 //cdn.dooca.store/158171/products/0jku3pabhtem1houcu6nvk7rjbkcz5q3ar6o.jpg?v=1722526922.
﻿Vinho Tinto Góes Tradição Bordô Suave Aromas intensos e paladar suave com a profundidade do rubi, ideal para carnes, massas e queijos.
19.2 //cdn.dooca.store/158171/products/stiosdee553pxevisvdlarz7tandsboelvuo.jpg?v=1723485548.
//www.vinicolagoes.com.br/vinho-tinto-quinta-jubair-bordo-demi-sec Vinho Tinto Quinta Jubair Bordô Demi-sec R$ 26,90 26.9 6890431 O Vinho Tinto de Mesa Quinta Jubair Bord&ocirc; Demi-sec &eacute; elaborado exclusivamente com a variedade Bord&ocirc;, resultando em um vinho de aspecto l&iacute;mpido e brilhante.
Este vinho tinto demi-sec &eacute; equilibrado, de toque meio seco, e apresen

In [130]:
prompt = f"""
<role>
{role}
</role>
<contexto>:
{contexto}
</contexto>
<pergunta>:
{pergunta}
</pergunta>
<resposta>:
...
"""




In [131]:
print(prompt)



<role>
Você é um sommelier experiente.
          trabalha em um renomado restaurante 5 estrelas
          Sua função é recomendar vinhos assertivos ao cliente
          voce acabou de receber uma descrição de vinho, e deve usar o contexto para construir uma recomendação apropriada,
          lembre que voce esta falando diretamente com o cliente, entao seja educado e profissional,
          também explique o porque o vinho escolhido é o melhor para o cliente, e como ele orna com o prato
          também forneça seu preço
</role>
<contexto>:
﻿Vinho Tinto Tempos de Góes Tannat Reserva 2023 Aromas de frutas vermelhas e especiarias, taninos firmes, ideal para grelhados e queijos fortes. É ideal para ser servido entre 16ºC a 18ºC, harmonizando perfeitamente com grelhados, carnes exóticas, queijos fortes e molhos condimentados. 94.08 //cdn.dooca.store/158171/products/0jku3pabhtem1houcu6nvk7rjbkcz5q3ar6o.jpg?v=1722526922. ﻿Vinho Tinto Góes Tradição Bordô Suave Aromas intensos e paladar suave

In [132]:
# wo_rag = get_answer(pergunta)


In [133]:
with_rag = get_answer(prompt)


In [134]:
# print("\nPergunta:")
# print(pergunta)

# print("\n====================\n")

# print("\nResposta sem RAG:")
# print(formatar_resposta(wo_rag))

# print("\n====================\n")

# print("Resposta com RAG:")
print(formatar_resposta(with_rag))


Em primeiro lugar, agradeço-vos por ter escolhido o nosso restaurante para esta experiência gastronômica.
Para o Stake Tartar que vos deleitareis hoje, eu recomendo o Vinho Tinto Tempos de Góes Tannat Reserva 2023.

O Tannat é um vinho tinto forte e robusto, que combina perfeitamente com carnes vermelhas, como o Stake Tartar, graças às suas notas taninosas e saborosas de frutas vermelhas.
Este vinho também se destaca por sua capacidade de harmonizar bem com molhos condimentados, o que torna ele a melhor opção para este prato.

A temperatura ideal para servir o Tempos de Góes Tannat Reserva 2023 é entre 16ºC e 18ºC, permitindo que você aproveite toda a complexidade dos aromas e sabor deste vinho excepcional.

Você pode encontrar este vinho na nossa loja por R$ 94,08.
É uma oportunidade de experimentar um vinho tinto de alta qualidade que ornará perfeitamente seu Stake Tartar.

Espero que vos deleite! Sinta-se à vontade para qualquer pergunta adicional ou para mais recomendações personal